In [ ]:
#Baixando as bibliotecas
!pip install PyPDF2
!pip install tabula-py
!pip install --upgrade gspread

In [ ]:
#Importação das bilbiotecas
import tabula
import PyPDF2
import warnings
import pandas as pd
import re
from google.colab import files, auth
import gspread_dataframe
import gspread
import google.auth
from oauth2client.client import GoogleCredentials
#Ignorar mensagem de depreciação
warnings.filterwarnings("ignore")

In [ ]:

# Get a GoogleCredentials object.
credentials, _ = google.auth.default()

# Authorize the client.
gc = gspread.authorize(credentials)

In [ ]:
#Ignorar mensagem de depreciação
warnings.filterwarnings("ignore")

#Carregando o arquivo pdf
lista_tabelas = tabula.read_pdf('004815222_20230828_20230830_SPOT.pdf', pages='all')

#Exibindo quantas páginas foram caregadas
print(len(lista_tabelas))

Jan 08, 2024 7:32:38 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



4


In [ ]:
#Gerando um loop for para exibir cada página do PDF
for tabela in lista_tabelas:
    display(tabela) #Display só funciona no notebook, no VSC deve usar o print

,Nr. nota\r43703437,Folha\r1,Data pregão\r28/08/2023
0,BTG Pactual CTVM S.A.\rAvenida Brigadeiro Fari...,NaN,NaN
1,Cliente\r004815222\rAUGUSTO GABRIEL RODRIGUES ...,NaN,C.P.F./C.N.P.J/C.V.M./C.O.B.\r067.962.941-65
2,NaN,Código cliente\r85-0 004815222,Assessor


,Participante destino do repasse\r1026,Unnamed: 0,Cliente\r-,Unnamed: 1,Valor\r0,Unnamed: 2,Custodiante,C.I\rN,Unnamed: 3,Unnamed: 4
0,Banco,Agência,Conta Corrente,NaN,Acionista,NaN,Administrador,NaN,Complemento nome,P.Vinc\rN


,Q,Negociação,C/V,Tipo Mercado,Prazo,Especificação do título,Obs. (*),Quantidade,Preço / Ajuste,Valor Operação / Ajuste,D/C
0,NaN,1-BOVESPA,V,VISTA,NaN,CYRE3FON,NaN,2,"23,37","46,74",C
1,NaN,1-BOVESPA,V,VISTA,NaN,ENAT3FON,NaN,1,"15,39","15,39",C
2,NaN,1-BOVESPA,V,VISTA,NaN,INTB3FON,NaN,1,"23,24","23,24",C


,Resumo dos Negócios,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,"Debêntures0,00\rVendas à vista85,37\rCompras à...",NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total CBLC,"85,35",C,NaN
2,NaN,NaN,"Bolsa\rTaxa de termo/opções0,00\rTaxa A.N.A.0,...",NaN,NaN,NaN
3,NaN,NaN,"Total Bovespa / Soma0,00\rCorretagem / Despesa...",NaN,NaN,NaN
4,Especificações diversas\rA coluna Q indica liq...,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,Total corretagem / Despesas,NaN,"14,92",D
6,(*) Observações\rA - Posição futuroT - Liquida...,NaN,Líquido para 30/08/2023,"70,43",C,NaN
7,Observações:\r(1) As operações a termo não são...,NaN,NaN,NaN,NaN,NaN


In [ ]:
    #Obtendo somente a pagina 1.
    tabela1 = lista_tabelas[0]
    tabela1 = tabela1.melt(var_name='Coluna', value_name='Valor')
    tabela1[['Titulo', 'Valor']] = tabela1['Coluna'].str.split('\r', expand=True)
    tabela1 = tabela1[['Titulo','Valor']]
    tabela1 = tabela1.iloc[[0,3,6]]

    display(tabela1)


,Titulo,Valor
0,Nr. nota,43703437
3,Folha,1
6,Data pregão,28/08/2023


In [ ]:
    tabela3 = lista_tabelas[2]
    tabela3

,Q,Negociação,C/V,Tipo Mercado,Prazo,Especificação do título,Obs. (*),Quantidade,Preço / Ajuste,Valor Operação / Ajuste,D/C
0,NaN,1-BOVESPA,V,VISTA,NaN,CYRE3FON,NaN,2,"23,37","46,74",C
1,NaN,1-BOVESPA,V,VISTA,NaN,ENAT3FON,NaN,1,"15,39","15,39",C
2,NaN,1-BOVESPA,V,VISTA,NaN,INTB3FON,NaN,1,"23,24","23,24",C


In [ ]:
# Supondo que 'lista_tabelas' seja uma lista de DataFrames
tabela4 = lista_tabelas[3]

# Selecionando apenas as linhas de interesse
tabela4 = tabela4.iloc[[0, 1, 2, 3, 5, 6]]
tabela4 = tabela4.drop(['Unnamed: 0', 'Unnamed: 4'], axis=1)

# Substituindo caracteres indesejados por espaço
tabela4[['Resumo dos Negócios', 'Unnamed: 1']] = tabela4[['Resumo dos Negócios', 'Unnamed: 1']].replace('[^0-9,]', ' ', regex=True)
tabela4[['Resumo dos Negócios', 'Unnamed: 1']] = tabela4[['Resumo dos Negócios', 'Unnamed: 1']].replace('\s+', ' ', regex=True)
tabela4[['Resumo dos Negócios', 'Unnamed: 1']] = tabela4[['Resumo dos Negócios', 'Unnamed: 1']].replace(' ', '/', regex=True)
tabela4

,Resumo dos Negócios,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,"/0,00/85,37/0,00/0,00/0,00/0,00/0,00/85,37",NaN,NaN,NaN
1,NaN,/,"85,35",C
2,NaN,"/0,00/0,00/0,00",NaN,NaN
3,NaN,"/0,00/13,50/0,00/0,00/1,42/,/0,000,00/0,00",NaN,NaN
5,NaN,/,NaN,"14,92"
6,/2/8/,/30/08/2023,"70,43",C


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Crie uma nova planilha
new_wb = gc.create('Nova_Planilha')

# Crie páginas para cada tabela
new_wb.add_worksheet(title='Tabela1', rows=1, cols=1)
new_wb.add_worksheet(title='Tabela3', rows=1, cols=1)
new_wb.add_worksheet(title='Tabela4', rows=1, cols=1)

# Acesse cada página
ws_tabela1 = new_wb.get_worksheet(0)
ws_tabela3 = new_wb.get_worksheet(1)
ws_tabela4 = new_wb.get_worksheet(2)

# Converta os valores para strings antes de atualizar
ws_tabela1.update('A1', tabela1.astype(str).values.tolist())
ws_tabela3.update('A1', tabela3.astype(str).values.tolist())
ws_tabela4.update('A1', tabela4.astype(str).values.tolist())


{'spreadsheetId': '17ZoMXqx7buVBM-oCpM1EJ818nTVF_9Ei-n6jvmLac-M',
 'updatedRange': 'Tabela3!A1:D6',
 'updatedRows': 6,
 'updatedColumns': 4,
 'updatedCells': 24}

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Abra a planilha existente pelo seu ID ou nome
existing_wb = gc.open_by_key('11-EGBySBtTnIEtUCT11oRGXkZGo0j21qugbPbkMMS8s')  # Substitua 'Sua_Chave' pelo ID da sua planilha

# Adicione novas páginas para cada tabela
ws_tabela1 = existing_wb.add_worksheet(title='Tabela1', rows=1, cols=1)
ws_tabela3 = existing_wb.add_worksheet(title='Tabela3', rows=1, cols=1)
ws_tabela4 = existing_wb.add_worksheet(title='Tabela4', rows=1, cols=1)

# Converta os valores para strings antes de atualizar
ws_tabela1.update('A1', tabela1.astype(str).values.tolist())
ws_tabela3.update('A1', tabela3.astype(str).values.tolist())
ws_tabela4.update('A1', tabela4.astype(str).values.tolist())


{'spreadsheetId': '11-EGBySBtTnIEtUCT11oRGXkZGo0j21qugbPbkMMS8s',
 'updatedRange': 'Tabela4!A1:D6',
 'updatedRows': 6,
 'updatedColumns': 4,
 'updatedCells': 24}